In [1]:
import os

In [2]:
%pwd

'C:\\Users\\shash\\fetch_assessment\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\shash\\fetch_assessment'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file:Path
    unzip_dir: Path

In [6]:
from fetch_assessment.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from fetch_assessment.utils.common import read_yaml, create_directories

Importing constants
Config path: config\config.yaml
Params path: params.yaml


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from fetch_assessment.logging import logger
from fetch_assessment.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    
except Exception as e:
    raise e

[2023-11-12 18:00:08,707: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-12 18:00:08,719: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-12 18:00:08,723: INFO: common: created directory at: artifacts]
[2023-11-12 18:00:08,727: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-12 18:00:10,225: INFO: 1235568240: artifacts/data_ingestion/data_daily.csv download! with following info: 
x-amz-id-2: UDhd+tMbVatRs4UgIYzhUZZdZ9NnvwfKyMYK4FBMCs0liCq2r2OcmL4kFYaJBkPNe3XdUKIKhNnGOFy2x0VMBA==
x-amz-request-id: 85Y15CBPRTHT1FQ6
Date: Mon, 13 Nov 2023 02:00:09 GMT
Last-Modified: Sat, 12 Nov 2022 03:32:30 GMT
ETag: "b5570f9c745a220559c5e9855b0802c7"
x-amz-version-id: 6NpGY2C9NAqRqlLXTZkVltuvfeatPE84
Accept-Ranges: bytes
Content-Type: text/csv
Server: AmazonS3
Content-Length: 6981
Connection: close

]
